In [171]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\kman9\appdata\local\programs\python\python39\lib\site-packages (1.5.3)



You should consider upgrading via the 'c:\Users\Kman9\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kman9\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [172]:
import pandas as pd
import numpy as np

In [173]:
# read all data
shg = pd.read_csv('data/suicide homicide gdp.csv')
ph = pd.read_csv('data/share-of-deaths-homicides.csv')
emis = pd.read_csv('data/GCB2022v27_MtCO2_flat.csv')
emis[:5]

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
ph.isnull().sum()

Entity                                                                     0
Code                                                                     690
Year                                                                       0
Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Percent)      0
dtype: int64

In [175]:
# drop rows with year < 1990
emis = emis[emis['Year'] >= 1990]
emis[:5]

# drop Other column
emis = emis.drop(columns = ['Other'])

In [176]:
# remove NaN values
emis.isnull().sum()

c = emis.groupby('Country')

c.apply(lambda x: x.isnull().sum())


,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Per Capita
Country,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,0,0,0,0,0,0,0
Andorra,0,0,0,0,0,0,0,0,0,0
Angola,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
Viet Nam,0,0,0,0,0,0,0,0,0,0
Wallis and Futuna Islands,0,0,0,0,0,0,0,11,0,0
Yemen,0,0,0,0,0,0,0,0,0,0


In [177]:
# set year column as index
emis.set_index('Year', inplace=True)

# fill missing values with previous and forward 5-year averages
emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)

# reset index
emis.reset_index(inplace=True)

emis.isnull().sum()

C:\Users\Kman9\AppData\Local\Temp\ipykernel_17064\1088784760.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Country', 'ISO 3166-1 alpha-3'], dtype='object')
  emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)


Year                    0
Country                 0
ISO 3166-1 alpha-3    192
Total                   0
Coal                    0
Oil                     0
Gas                     0
Cement                  0
Flaring                 0
Per Capita              0
dtype: int64

In [178]:
# merging tables
merged = pd.merge(ph, emis, )

KeyError: 'Country'